In [1]:
import os
from time import sleep
import xarray as xr
import pandas as pd
from tqdm.notebook import tqdm
from datetime import timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from pylibs.plot_utils import set_size
from pylibs.setuparea import setarea
from pylibs.utils import get_dates

In [2]:
tlsize = 12
txsize = 12
mpl.rc('axes', titlesize=tlsize, labelsize=txsize)
mpl.rc('xtick', labelsize=txsize)
mpl.rc('ytick', labelsize=txsize)
mpl.rc('legend', fontsize='small')
axe_w = 4; axe_h = 4; plot_quality = 600

In [3]:
srcpath = '/glade/work/swei/Git/JEDI-METplus'
area = 'wxaq'
minlon = -81.; maxlon = -70; minlat = 39.8; maxlat = 46.

plot_product = 'wrfchem_evaluate'
obs_name = 'airnow-wxaq'

vfhr_range = range(1, 25)
init_sdate = 2024082200
init_edate = 2024083100
hint = 24
init_dates = get_dates(init_sdate, init_edate, hint)

unit_str = 'mol m$^{-2}$'

hofx_path_tmpl = os.path.join(srcpath, 'output', plot_product, 'hofx', 'f{fhr:02d}', obs_name)
plts_path = os.path.join(srcpath, 'output', plot_product, 'plots', '2dmap', obs_name)

if not os.path.exists(plts_path):
    os.makedirs(plts_path)

# Setup projection
if area == 'wxaq':
    proj = ccrs.LambertConformal(central_longitude=-97.0,
                                 central_latitude=39.0,
                                 standard_parallels=[30., 60.])
if area == 'glb':
    proj = ccrs.PlateCarree()

In [4]:
df = pd.DataFrame()
pbardates = tqdm(init_dates, desc=f'Starting')
for idate in pbardates:
    pbardates.set_description(f'Processing {idate}')
    for fhr in vfhr_range:
        hofx_path = hofx_path_tmpl.format(fhr=fhr)
        if not os.path.exists(hofx_path):
            raise Exception(f'HofX folder: {hofx_path} is not available')
        cdate = idate + timedelta(hours=fhr)
        plot_date = cdate.strftime('%Y%m%d%H')
        hofx_file = f'hofx.{obs_name}.{plot_date}.nc4'
        in_hofx = os.path.join(hofx_path, hofx_file)
        if not os.path.exists(in_hofx):
            print(f'WARNING: Skip {plot_date}, {hofx_file} is not available')
            continue

        meta_ds = xr.open_dataset(in_hofx, group='MetaData')
        obsv_ds = xr.open_dataset(in_hofx, group='ObsValue')
        tmpdf = meta_ds.to_dataframe()
        tmpobsdf = obsv_ds.to_dataframe()
        tmpdf = pd.concat((tmpdf, tmpobsdf), axis=1)
        
        df = pd.concat((df, tmpdf))

Starting:   0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
ozone_df = df[['latitude', 'longitude', 'stationIdentification', 'ozoneInsitu']]
pm2p5_df = df[['latitude', 'longitude', 'stationIdentification', 'particulatematter2p5Insitu']]
# no2_df = df[['latitude', 'longitude', 'stationIdentification', 'nitrogendioxideInsitu']]

In [24]:
ploto3_df = ozone_df.groupby(['stationIdentification']).agg(func='mean').dropna(subset=['ozoneInsitu'])
plotpm_df = pm2p5_df.groupby(['stationIdentification']).agg(func='mean').dropna(subset=['particulatematter2p5Insitu'])
# plotno2df = no2_df.groupby(['stationIdentification']).agg(func='mean').dropna(subset=['nitrogendioxideInsitu'])
overlap_df = pd.merge(ploto3_df, plotpm_df, on=['longitude', 'latitude'])

In [25]:
o3_locations = ploto3_df[['longitude', 'latitude']].drop_duplicates()
pm_locations = plotpm_df[['longitude', 'latitude']].drop_duplicates()
# overlap_df = pd.merge(o3_locations, pm_locations, on=['longitude', 'latitude'], how='inner', indicator=True)
# overlapping = overlap_df[['longitude', 'latitude']]
# o3_only_df = o3_locations.merge(pm_locations, on=['longitude', 'latitude'], how='left', indicator=True).query('_merge == "left_only"')[['longitude', 'latitude']]
# pm_only_df = pm_locations.merge(o3_locations, on=['longitude', 'latitude'], how='left', indicator=True).query('_merge == "left_only"')[['longitude', 'latitude']]

In [48]:
fig = plt.figure()
ax = plt.subplot(projection=proj)
set_size(axe_w, axe_h, t=0.96, l=0.14, b=0.1, r=0.86)
ax.set_extent((minlon, maxlon, minlat, maxlat), crs=ccrs.PlateCarree())
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
gl.xlines = False
gl.ylines = False
gl.xlabel_style = {'size': txsize - 1}
gl.ylabel_style = {'size': txsize - 1}
gl.right_labels = False
gl.top_labels = False
ax.add_feature(cft.BORDERS.with_scale('10m'), color='grey', linestyle='--', linewidth=0.5)
ax.add_feature(cft.STATES.with_scale('10m'), edgecolor='grey', linewidth=0.5)
ax.add_feature(cft.LAKES.with_scale('10m'), facecolor='None', edgecolor='grey', linewidth=0.5)

ax.scatter(o3_locations['longitude'], o3_locations['latitude'], c='none', s=50,
           edgecolor='k', linewidth=1, transform=ccrs.PlateCarree(), zorder=2, label='$\mathrm{O}_3$')
ax.scatter(pm_locations['longitude'], pm_locations['latitude'], c='b', s=15,
           edgecolor='none', transform=ccrs.PlateCarree(), zorder=3, label='$\mathrm{PM}_{2.5}$')

ax.legend(loc='lower right')

plotname = f'site_maps.{init_sdate}_{init_edate}.png'
outname = os.path.join(plts_path, plotname)
# plt.tight_layout()
fig.savefig(outname, dpi=plot_quality)
plt.close()

/glade/u/apps/opt/conda/envs/npl-2025b/lib/python3.12/site-packages/cartopy/mpl/feature_artist.py:144: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '


In [35]:
outname

'/glade/work/swei/Git/JEDI-METplus/output/wrfchem_evaluate/plots/2dmap/airnow-wxaq/site_maps.2024082200_2024083100.png'